In [1]:
import pandas as pd

In [18]:
df = pd.read_csv("processed_df.csv")

In [19]:
df.head(3)

,Unnamed: 0,order_date,delivery_date,payment_type,price,discount_percent,quantity,shipping_region,expected_delivery_days,product_rating,product_category,product_subcategory,brand,loyalty_points,is_first_order,device_type,referral,join_date,age,gender,location,preferred_payment,total_orders,total_returns,avg_order_value,product_name,avg_rating,return_rate_category,return_reason,is_returned,order_month,order_year,order_dayofweek,discount,revenue,customer_age_group,is_expensive
0,0,2024-03-08,2024-03-12,DebitCard,1413.25,0.72,3,West,4,2.8,Apparel,Shoes,Puma,429,True,Desktop,CampaignB,2024-09-11,64,Female,Chennai,UPI,38,16,1594.18,Puma Shoes,4.5,0.25,Defective,1,3,2024,Fri,10.18,4209.21,60+,1
1,1,2024-10-05,2024-10-08,CreditCard,328.40,20.15,1,North,3,1.3,Apparel,Shoes,Samsung,338,False,Mobile,Organic,2020-03-27,59,Male,Hyderabad,Wallet,9,10,584.09,Samsung Shoes,3.0,0.12,Not Returned,0,10,2024,Sat,66.17,262.23,40-59,0
2,2,2023-10-22,2023-10-24,CreditCard,3366.84,49.49,3,West,2,3.9,Home,Kitchen,L'Oreal,416,False,Tablet,Organic,2024-05-16,58,Female,Hyderabad,COD,67,9,2430.24,L'Oreal Kitchen,4.3,0.18,Not Returned,0,10,2023,Sun,1666.25,5101.77,40-59,1


In [20]:
df.drop(["Unnamed: 0"], axis=1, inplace=True)

In [21]:
pd.set_option('display.max_columns', None)
df.head(2)

,order_date,delivery_date,payment_type,price,discount_percent,quantity,shipping_region,expected_delivery_days,product_rating,product_category,product_subcategory,brand,loyalty_points,is_first_order,device_type,referral,join_date,age,gender,location,preferred_payment,total_orders,total_returns,avg_order_value,product_name,avg_rating,return_rate_category,return_reason,is_returned,order_month,order_year,order_dayofweek,discount,revenue,customer_age_group,is_expensive
0,2024-03-08,2024-03-12,DebitCard,1413.25,0.72,3,West,4,2.8,Apparel,Shoes,Puma,429,True,Desktop,CampaignB,2024-09-11,64,Female,Chennai,UPI,38,16,1594.18,Puma Shoes,4.5,0.25,Defective,1,3,2024,Fri,10.18,4209.21,60+,1
1,2024-10-05,2024-10-08,CreditCard,328.40,20.15,1,North,3,1.3,Apparel,Shoes,Samsung,338,False,Mobile,Organic,2020-03-27,59,Male,Hyderabad,Wallet,9,10,584.09,Samsung Shoes,3.0,0.12,Not Returned,0,10,2024,Sat,66.17,262.23,40-59,0


In [31]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

In [ ]:
def build_pipeline(numeric_cols, categorical_ohe_cols):
    # numeric pipeline
    num_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])
    cat_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="frequency")),
        ("onehot", OneHotEncoder(handle_unknown="ignore", sparse=False))
    ])
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", num_pipeline, numeric_cols),
            ("cat", cat_pipeline, categorical_ohe_cols)
        ],
    )
    # classifier
    clf = LGBMClassifier(
        objective="binary",
        n_jobs=-1,
        random_state=42,
        n_estimators=500,
        is_unbalance=True  # helps with class imbalance
    )
    pipeline = Pipeline([
        ("preproc", preprocessor),
        ("clf", clf)
    ])
    return pipeline

In [23]:
date_col = ['order_date','delivery_date','join_date']

for col in date_col:
    df[col] = pd.to_datetime(df[col])

In [25]:
obj_col = ['is_returned','order_month','order_year','is_expensive']

for col in obj_col:
    df[col] = df[col].astype('object')

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 36 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   order_date              20000 non-null  datetime64[ns]
 1   delivery_date           20000 non-null  datetime64[ns]
 2   payment_type            20000 non-null  object        
 3   price                   20000 non-null  float64       
 4   discount_percent        20000 non-null  float64       
 5   quantity                20000 non-null  int64         
 6   shipping_region         20000 non-null  object        
 7   expected_delivery_days  20000 non-null  int64         
 8   product_rating          20000 non-null  float64       
 9   product_category        20000 non-null  object        
 10  product_subcategory     20000 non-null  object        
 11  brand                   20000 non-null  object        
 12  loyalty_points          20000 non-null  int64 

In [34]:
X = df.drop("is_returned", axis=1)
y = df['is_returned']

In [56]:
numerical_cols = X.select_dtypes(include=['int32', 'float32','float64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

In [57]:
print(numerical_cols)
print(categorical_cols)


['price', 'discount_percent', 'product_rating', 'avg_order_value', 'avg_rating', 'return_rate_category', 'discount', 'revenue']
['payment_type', 'shipping_region', 'product_category', 'product_subcategory', 'brand', 'device_type', 'referral', 'gender', 'location', 'preferred_payment', 'product_name', 'return_reason', 'order_month', 'order_year', 'order_dayofweek', 'customer_age_group', 'is_expensive']


In [58]:
num_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])
cat_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("labelencoder", OneHotEncoder(handle_unknown='ignore'))
    ])
preprocessor = ColumnTransformer(
        transformers=[
            ("num", num_pipeline, numerical_cols),
            ("cat", cat_pipeline, categorical_cols)
        ],
    )

In [59]:
X_transformed = preprocessor.fit_transform(X)

In [86]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_transformed, y, test_size=0.25, random_state=42, stratify=y
)


In [87]:
print("X_train -> ",X_train.shape )
print("X_test -> ",X_test.shape )
print("y_train -> ",y_train.shape )
print("y_test -> ",y_test.shape )

X_train ->  (15000, 306)
X_test ->  (5000, 306)
y_train ->  (15000,)
y_test ->  (5000,)


In [88]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)


In [89]:
# y_train.unique()
# y_test.unique()

In [90]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [91]:
y_pred = lr.predict(X_test)

In [94]:
from sklearn.metrics import classification_report, accuracy_score
y_pred = lr.predict(X_test)
print("Accuracy:", classification_report(y_test, y_pred))


Accuracy:               precision    recall  f1-score   support

           0       1.00      1.00      1.00      4000
           1       1.00      1.00      1.00      1000

    accuracy                           1.00      5000
   macro avg       1.00      1.00      1.00      5000
weighted avg       1.00      1.00      1.00      5000



In [95]:
from lightgbm import LGBMClassifier
clf = LGBMClassifier(
        objective="binary",
        n_jobs=-1,
        random_state=42,
        n_estimators=500,
        is_unbalance=True  # helps with class imbalance
    )

In [96]:
clf.fit(X_train,y_train)

[LightGBM] [Info] Number of positive: 3000, number of negative: 12000
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003061 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1954
[LightGBM] [Info] Number of data points in the train set: 15000, number of used features: 302
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.200000 -> initscore=-1.386294
[LightGBM] [Info] Start training from score -1.386294
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.1
,n_estimators,500
,subsample_for_bin,200000
,objective,'binary'
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [98]:
y_pred_clf = clf.predict(X_test)

c:\Users\TransOrg\Case Studies\END-TO-END\E-commerce-Product-Return-Prediction\prod_return_env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [99]:
print("Accuracy:", classification_report(y_test, y_pred_clf))

Accuracy:               precision    recall  f1-score   support

           0       1.00      1.00      1.00      4000
           1       1.00      1.00      1.00      1000

    accuracy                           1.00      5000
   macro avg       1.00      1.00      1.00      5000
weighted avg       1.00      1.00      1.00      5000

